In [2]:
#imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as m
import os


#Feel free to delete these if you don't want to install them all right away
import xgboost as xgb
from catboost import CatBoostClassifier
#Used sm and smf instead to get these, but can still use them if needed, but sm has better summary() function
from sklearn.linear_model import LinearRegression, LogisticRegression, TweedieRegressor
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import poisson, binom, norm, lognorm
from scipy.stats import chisquare
import statsmodels.api as sm
import statsmodels.formula.api as smf



from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, make_scorer, roc_auc_score, log_loss, precision_score, recall_score, roc_curve, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from collections import Counter
import scipy.stats as st



#Change this to whereever you save your stuff
os.chdir("C:\ASNA_Case_Comp")


<>:36: SyntaxWarning: invalid escape sequence '\A'
<>:36: SyntaxWarning: invalid escape sequence '\A'
C:\Users\graha\AppData\Local\Temp\ipykernel_29608\509306306.py:36: SyntaxWarning: invalid escape sequence '\A'
  os.chdir("C:\ASNA_Case_Comp")


In [3]:
#Read in dataset
dat = pd.read_excel("Dataset.xlsx")
dat_one_hot = pd.read_csv("cleanData.csv",header=0)

#Add a column for has claim
dat['has_claim'] = dat['claim_id'].apply(lambda x: 1 if x !=0 else 0)
dat_one_hot['has_claim'] = dat_one_hot['claim_id'].apply(lambda x: 1 if x !=0 else 0)

count = 0
for row in dat_one_hot['has_claim']:
    if row > 0:
        count+=1
print(count)
#There is this many total claims out of 10000 students

1819


# Reading in Aidan's data segmented by coverage type

In [4]:
dat_expense = pd.read_csv("CleanByCoverage\CleanByCoverage\expenseClean.csv",header=0)
dat_expense['has_claim'] = dat_expense['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_liability = pd.read_csv("CleanByCoverage\CleanByCoverage\liabilityClean.csv",header=0)
dat_liability['has_claim'] = dat_liability['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_medical = pd.read_csv("CleanByCoverage\CleanByCoverage\medicalClean.csv",header=0)
dat_medical['has_claim'] = dat_medical['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_property = pd.read_csv("CleanByCoverage\CleanByCoverage\propertyClean.csv",header=0)
dat_property['has_claim'] = dat_property['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_expense_original = pd.read_csv("OriginalByCoverage\OriginalByCoverage\expense.csv",header=0)
dat_expense_original['has_claim'] = dat_expense_original['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_liability_original = pd.read_csv("OriginalByCoverage\OriginalByCoverage\liability.csv",header=0)
dat_liability_original['has_claim'] = dat_liability_original['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_medical_original = pd.read_csv("OriginalByCoverage\OriginalByCoverage\medical.csv",header=0)
dat_medical_original['has_claim'] = dat_medical_original['claim_id'].apply(lambda x: 1 if x !=0 else 0)

dat_property_original = pd.read_csv("OriginalByCoverage\OriginalByCoverage\property.csv",header=0)
dat_property_original['has_claim'] = dat_property_original['claim_id'].apply(lambda x: 1 if x !=0 else 0)


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:7: SyntaxWarning: invalid escape sequence '\C'
<>:10: SyntaxWarning: invalid escape sequence '\C'
<>:13: SyntaxWarning: invalid escape sequence '\O'
<>:16: SyntaxWarning: invalid escape sequence '\O'
<>:19: SyntaxWarning: invalid escape sequence '\O'
<>:22: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:7: SyntaxWarning: invalid escape sequence '\C'
<>:10: SyntaxWarning: invalid escape sequence '\C'
<>:13: SyntaxWarning: invalid escape sequence '\O'
<>:16: SyntaxWarning: invalid escape sequence '\O'
<>:19: SyntaxWarning: invalid escape sequence '\O'
<>:22: SyntaxWarning: invalid escape sequence '\O'
C:\Users\graha\AppData\Local\Temp\ipykernel_29608\766309216.py:1: SyntaxWarning: invalid escape sequence '\C'
  dat_expense = pd.read_csv("CleanByCoverage\CleanByCoverage\expenseClean.csv",h

# Calculating Claim Frequency

In [5]:
print('Frequency for Additional Expense Coverage')
claim_frequency_expense = list(zip(dat_expense_original['has_claim'], dat_expense_original['risk_tier']))
claim_frequency_expense_counts = Counter(claim_frequency_expense)
for item, count in claim_frequency_expense_counts.items():
    print(f'{item}: {count}')

print('Frequency for Liability Coverage')
claim_frequency_liability = list(zip(dat_liability_original['has_claim'], dat_liability_original['risk_tier']))
claim_frequency_liability_counts = Counter(claim_frequency_liability)
for item, count in claim_frequency_liability_counts.items():
    print(f'{item}: {count}')

print('Frequency for Medical Coverage')
claim_frequency_medical = list(zip(dat_medical_original['has_claim'], dat_medical_original['risk_tier']))
claim_frequency_medical_counts = Counter(claim_frequency_medical)
for item, count in claim_frequency_medical_counts.items():
    print(f'{item}: {count}')

print('Frequency for Property Coverage')
claim_frequency_property = list(zip(dat_property_original['has_claim'], dat_property_original['risk_tier']))
claim_frequency_property_counts = Counter(claim_frequency_property)
for item, count in claim_frequency_property_counts.items():
    print(f'{item}: {count}')

Frequency for Additional Expense Coverage
(0, 2): 2100
(0, 3): 3124
(0, 1): 4264
(1, 2): 111
(1, 1): 255
(1, 3): 162
Frequency for Liability Coverage
(0, 2): 2190
(0, 3): 3251
(0, 1): 4474
(1, 3): 28
(1, 2): 19
(1, 1): 38
Frequency for Medical Coverage
(0, 2): 2154
(0, 3): 3209
(0, 1): 4408
(1, 3): 72
(1, 2): 55
(1, 1): 106
Frequency for Property Coverage
(0, 2): 1999
(0, 3): 3011
(0, 1): 4068
(1, 1): 470
(1, 3): 281
(1, 2): 222


# Modelling Claim Severity

In [6]:
total_loss_expense = dat_expense['amount'].sum()
print(f"Total Expense Loss: {total_loss_expense:,.2f}")

total_claims_expense = dat_expense['has_claim'].sum()
print(f"Total Expense Claims: {total_claims_expense}")

severity_expense = total_loss_expense / total_claims_expense
print(f"Expense Severity (Loss per Claim): {severity_expense:,.2f}")

total_loss_property = dat_property['amount'].sum()
print(f"Total Property Loss: {total_loss_property:,.2f}")

total_claims_property = dat_property['has_claim'].sum()
print(f"Total Property Claims: {total_claims_property}")

severity_property = total_loss_property / total_claims_property
print(f"Property Severity (Loss per Claim): {severity_property:,.2f}")

total_loss_medical = dat_medical['amount'].sum()
print(f"Total Medical Loss: {total_loss_medical:,.2f}")

total_claims_medical = dat_medical['has_claim'].sum()
print(f"Total Medical Claims: {total_claims_medical}")

severity_medical = total_loss_medical / total_claims_medical
print(f"Medical Severity (Loss per Claim): {severity_medical:,.2f}")

total_loss_liability = dat_liability['amount'].sum()
print(f"Total Liability Loss: {total_loss_liability:,.2f}")

total_claims_liability = dat_liability['has_claim'].sum()
print(f"Total Liability Claims: {total_claims_liability}")

severity_liability = total_loss_liability / total_claims_liability
print(f"Liability Severity (Loss per Claim): {severity_liability:,.2f}")




Total Expense Loss: 2,911,317.09
Total Expense Claims: 528
Expense Severity (Loss per Claim): 5,513.86
Total Property Loss: 995,460.03
Total Property Claims: 973
Property Severity (Loss per Claim): 1,023.08
Total Medical Loss: 3,870,552.10
Total Medical Claims: 233
Medical Severity (Loss per Claim): 16,611.81
Total Liability Loss: 737,410.81
Total Liability Claims: 85
Liability Severity (Loss per Claim): 8,675.42


# Stress Test: 1000 more people in greek residences ~ probability rise by 10%

In [7]:
#We have previously shown that likelihood that person has sprinkler follows a binomial p=0.7
#We have also shown claims accross all types of insurance follow some sort of lognormal

#Steps (Done using a Monte Carlo Simulation)
#I have calculated all the means, variances and probabilities in Case_Comp_Modelling, simply saving them here for easy use

#Step 1:
#Simulate 20% increase in people off-campus (Binomial Random Sample ~p=0.16 regardless of types)
greek_binomial_p = 0.197
#We can add 0.1 because the dataset is 10000 people, and I want to simulate a 1000 person increase
greek_binomial_p_new = greek_binomial_p + 0.1


#Step 2:
#Simulate whether the person has a claim or not (Binomial Random Sample, different P for each coverage type)

#[greek probability of claim, no greek probability of claim]
coverage_binomial_p = {"Expense": [0.0947,0.0424],
                       "Property": [0.1669,0.0795],
                       "Medical": [0.0361,0.0202],
                       "Liability": [0.0178,0.0062]}

#Step 3:
#Simulate Claim Amounts (Different Lognormal for each coverage type) [mu,std]

#[greek [mu,sigma], no greek [mu,sigma]]
coverage_lognormal_params = {"Expense": [[8.792251500587787,0.7860062738715917],[8.030335656316481,0.6733062976959585]],
                             "Property": [[7.069795665992642,0.7414021940612359],[6.42011980277624,0.7083362835890147]],
                             "Medical": [[9.75170860304146,1.0449930588238456],[8.979976322517073,0.9893257823172484]],
                             "Liability": [[8.873775404507777,1.067439697992325],[8.425258924450155,1.0886178366432073]]}

#Step 4: Calculate final Values for Frequency, Severity, Expected Loss(E[Loss]) and Probable Maximum Loss(PML)

np.random.seed(123)
#Here is the code to do the simulation
N_people = 10000
N_simulations = 5000

# Store results for all four coverage types
results = {cov: {"frequency": [], "severity": [], "EL": [], "PML": [], "capital_efficiency": []}for cov in coverage_binomial_p.keys()}

#Iterate through 5000 simulations
for _ in range(N_simulations):

    # Step 1: simulate binomial for increased off-campus presence
    offcampus = np.random.binomial(1, greek_binomial_p_new, N_people)

    #Iterating over the four types of claims
    for cov in coverage_binomial_p:

        #Fetch the probabilities for off and on campus, recall they are stored as [probability given off campus, probability given on campus]
        p_off, p_on = coverage_binomial_p[cov]
        (mu_off, sig_off), (mu_on, sig_on) = coverage_lognormal_params[cov]

        #Get indices where claims are off and on campus
        off_idx = np.where(offcampus == 1)[0]
        on_idx  = np.where(offcampus == 0)[0]

        #Calculate whether they have a claim based on whether they are off or on campus
        has_claim = np.zeros(N_people, dtype=int)
        has_claim[off_idx] = np.random.binomial(1, p_off, size=len(off_idx))
        has_claim[on_idx]  = np.random.binomial(1, p_on,  size=len(on_idx))

        #Calculate severity for off and on campus using the lognorm
        severity_vals = np.zeros(N_people)

        #Off-campus random value generation from off capmus lognorm
        off_claim_idx = off_idx[has_claim[off_idx] == 1]
        if len(off_claim_idx) > 0:
            severity_vals[off_claim_idx] = lognorm(s=sig_off, scale=np.exp(mu_off)).rvs(len(off_claim_idx))

        #On-campus random value generation from on campus lognorm
        on_claim_idx = on_idx[has_claim[on_idx] == 1]
        if len(on_claim_idx) > 0:
            severity_vals[on_claim_idx] = lognorm(s=sig_on, scale=np.exp(mu_on)).rvs(len(on_claim_idx))

        #Calculate The metrics from these values
        freq = has_claim.mean()

        sev = severity_vals[severity_vals > 0].mean() if has_claim.sum() > 0 else 0

        EL = severity_vals.sum() / N_people

        PML = np.percentile(severity_vals, 99)

        capital_efficiency = EL / PML

        #Store them in results dictionary
        results[cov]["frequency"].append(freq)
        results[cov]["severity"].append(sev)
        results[cov]["EL"].append(EL)
        results[cov]["PML"].append(PML)
        results[cov]["capital_efficiency"].append(capital_efficiency)

# Final summary
summary = []
for cov in results:
    summary.append({
        "Coverage": cov,
        "Frequency": np.mean(results[cov]["frequency"]),
        "Severity": np.mean(results[cov]["severity"]),
        "Expected_Loss": np.mean(results[cov]["EL"]),
        "PML_99": np.mean(results[cov]["PML"]),
        "Capital_Efficiency": np.mean(results[cov]["capital_efficiency"])
    })

summary_df = pd.DataFrame(summary)
summary_df

C:\Users\graha\AppData\Local\Temp\ipykernel_29608\2657103005.py:86: RuntimeWarning: divide by zero encountered in scalar divide
  capital_efficiency = EL / PML


,Coverage,Frequency,Severity,Expected_Loss,PML_99,Capital_Efficiency
0,Expense,0.057910,6336.773687,366.961428,9708.745316,0.037829
1,Property,0.105385,1145.821943,120.751050,2380.204029,0.050769
2,Medical,0.024934,20132.670850,502.023272,14323.787789,0.035181
3,Liability,0.009633,10617.385487,102.248532,340.162902,inf


In [8]:
def calculate_stats(dat_input, coverage, N_simulations=50_000, pml_quantile=0.99):

    # --- Clean severity data ---
    severity_data = dat_input['amount']
    severity_data = severity_data.replace([np.inf, -np.inf], np.nan)
    severity_data = severity_data[severity_data > 0].dropna()

    # --- Fit lognormal ---
    sigma, loc, scale = st.lognorm.fit(severity_data, floc=0)
    mu = np.log(scale)

    # --- Frequency ---
    exposures = dat_input[['student_id']].drop_duplicates()
    frequency = dat_input['has_claim'].sum() / len(exposures)

    # --- Severity (lognormal mean) ---
    severity = np.exp(mu + 0.5 * sigma**2)

    # --- Expected Loss per Person ---
    expected_loss = frequency * severity

    # --- Monte Carlo for *per-person* loss ---
    # 1. Claim indicator for each simulation: 0 or 1
    claim_indicator = np.random.binomial(1, frequency, size=N_simulations)

    # 2. A severity draw for each simulation
    severity_draws = np.random.lognormal(mean=mu, sigma=sigma, size=N_simulations)

    # 3. Per-person simulated losses
    losses = claim_indicator * severity_draws

    # --- Per-Person PML ---
    pml = np.percentile(losses, pml_quantile * 100)

    # Package results
    results = {
        "Coverage": coverage,
        "Frequency": frequency,
        "Severity": severity,
        "Expected_Loss": expected_loss,
        "PML_99": pml
    }

    results_df = pd.DataFrame([results])
    return results_df


In [9]:
np.random.seed(123)

results_expense_df = calculate_stats(dat_expense, 'Expense')
results_property_df = calculate_stats(dat_property, 'Property')
results_medical_df = calculate_stats(dat_medical, 'Medical')
results_liability_df = calculate_stats(dat_liability, 'Liability')

df_all_results = pd.concat([results_expense_df, results_property_df, results_medical_df, results_liability_df], ignore_index=True)
df_all_results


,Coverage,Frequency,Severity,Expected_Loss,PML_99
0,Expense,0.0528,5554.517874,293.278544,8064.856122
1,Property,0.0973,1041.362050,101.324528,2038.790459
2,Medical,0.0233,17762.891693,413.875376,11793.021351
3,Liability,0.0085,10071.334655,85.606345,0.000000
